In [4]:
import os
import re
import sys
import subprocess
from stdlib_list import stdlib_list

# Add a mapping of import names to package names
IMPORT_TO_PACKAGE_MAP = {
    'yaml': 'PyYAML',
    'ruamel': 'ruamel.yaml',
}

def find_imports(root_dir):
    imports = set()

    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.py'):
                filepath = os.path.join(root, file)
                with open(filepath, 'r', encoding='utf-8') as f:
                    content = f.read()
                    import_matches = re.findall(r'^\s*import\s+([\w.]+)', content, re.MULTILINE)
                    from_import_matches = re.findall(r'^\s*from\s+([\w.]+)\s+import', content, re.MULTILINE)
                    imports.update(match.split('.')[0] for match in import_matches if not match.startswith('.'))
                    imports.update(match.split('.')[0] for match in from_import_matches if not match.startswith('.'))

    return imports

def get_installed_version(package):
    try:
        result = subprocess.run(['pip', 'show', package], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode == 0:
            version_line = re.search(r'^Version: (.+)$', result.stdout, re.MULTILINE)
            if version_line:
                return version_line.group(1)
    except Exception as e:
        print(f"Error retrieving version for {package}: {e}")
    return None

def generate_requirements(imports):
    python_version = ".".join(map(str, sys.version_info[:2]))
    standard_libs = set(stdlib_list(python_version))

    requirements = []
    for package in sorted(imports):
        if package not in standard_libs:
            # Map the import name to the package name if necessary
            package_name = IMPORT_TO_PACKAGE_MAP.get(package, package)

            version = get_installed_version(package_name)
            if version:
                requirements.append(f"{package_name}=={version}")
            else:
                requirements.append(package_name)
    return requirements

In [5]:
root_dir = os.getcwd()
imports = find_imports(root_dir)
requirements = generate_requirements(imports)
print("Generated requirements.txt:")
print("\n".join(requirements))

Generated requirements.txt:
gdown==4.6.0
monai==1.1.0
numpy==1.21.2
pandas==1.4.1
plug_ai==23.3a0
requests==2.27.1
ruamel.yaml==0.17.21
setuptools==59.5.0
torch==1.10.0a0+git302ee7b
torchmed==0.0.1a0
PyYAML==5.4.1


In [7]:
with open('requirements.txt', 'w') as f:
    f.write('\n'.join(requirements))